## Libraries

In [41]:
import json
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from textblob import TextBlob
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from textblob import TextBlob
import os
from autocorrect import Speller


## Load JSON File

The JSON file below contains all the intents and corresponding sentence patterns that will be used to train the data. 

In [42]:
data = json.loads(open('data.json').read())

In [43]:
intents = data['interactionModel']['languageModel']
intents

{'intent': [{'name': 'hello',
   'samples': ['Hi', 'Hello', 'hey', 'hola', 'ey', 'oi', 'ola', 'saludos']},
  {'name': 'feedback',
   'samples': ['feedback',
    'I want some feedback',
    'Give me feedback',
    'I want feedback',
    'realimentación',
    'quiero darte retroalimentación',
    'retroalimentación',
    'quiero dejarte un mensaje']},
  {'name': 'cancel', 'samples': ['cancel', 'cancel it', 'cancelar']},
  {'name': 'chicken',
   'samples': ['arthur',
    'chicken',
    'I want arthur',
    'I want the chicken',
    'Bring the chicken',
    'about Arthur',
    'galinha',
    'pollo']},
  {'name': 'dashboard',
   'samples': ['dashboard',
    'Show me the dashboard',
    'Bring the dashboard',
    'I want the dashboard',
    'about dashboard',
    'tablero',
    'tablero de administración',
    'panel de administración',
    'llévame al panel de administración']},
  {'name': 'datetime',
   'samples': ['datetime',
    'time',
    'tell me the time',
    'show the time',
    '

In [44]:
# Create a list of tuples that assigns a list of words to an intent
words = []
labels = []
documents = []
punc = ['?', '!', ',', '.']

for intent in intents['intent']: 
    for sample in intent['samples']:
        word_list = nltk.word_tokenize(sample)
        words.extend(word_list)
        documents.append((word_list, intent['name']))
        if intent['name'] not in labels:
            labels.append(intent['name'])
documents

[(['Hi'], 'hello'),
 (['Hello'], 'hello'),
 (['hey'], 'hello'),
 (['hola'], 'hello'),
 (['ey'], 'hello'),
 (['oi'], 'hello'),
 (['ola'], 'hello'),
 (['saludos'], 'hello'),
 (['feedback'], 'feedback'),
 (['I', 'want', 'some', 'feedback'], 'feedback'),
 (['Give', 'me', 'feedback'], 'feedback'),
 (['I', 'want', 'feedback'], 'feedback'),
 (['realimentación'], 'feedback'),
 (['quiero', 'darte', 'retroalimentación'], 'feedback'),
 (['retroalimentación'], 'feedback'),
 (['quiero', 'dejarte', 'un', 'mensaje'], 'feedback'),
 (['cancel'], 'cancel'),
 (['cancel', 'it'], 'cancel'),
 (['cancelar'], 'cancel'),
 (['arthur'], 'chicken'),
 (['chicken'], 'chicken'),
 (['I', 'want', 'arthur'], 'chicken'),
 (['I', 'want', 'the', 'chicken'], 'chicken'),
 (['Bring', 'the', 'chicken'], 'chicken'),
 (['about', 'Arthur'], 'chicken'),
 (['galinha'], 'chicken'),
 (['pollo'], 'chicken'),
 (['dashboard'], 'dashboard'),
 (['Show', 'me', 'the', 'dashboard'], 'dashboard'),
 (['Bring', 'the', 'dashboard'], 'dashboard'

### Natural Language Processing

To prepare the text date for the model, the words in the sentence patterns will undergo some preprocessing. Firstly, the sentence phrases will all be turned to lower case. Next, punctuation will be eliminated as it has no influence over the intent of a query. Finally, using an NLP library, each word will be converted to its "base" form. This will ensure that users can ask for something in different ways and still get the same response. 

In [45]:
# import NLP library
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()
words = [lem.lemmatize(word.lower()) for word in words if word not in punc]

# sort words and remove duplicates
words = sorted(set(words))
labels = sorted(set(labels))


# Create files storing words and labels for personal reference
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(words, open('labels.pkl', 'wb'))


## Bag of Words

The text data has to be converted into matrices before it can be inputted into a deep learning model. Here, the text data for each intent is represented by a bag of words. Simply put, the frequency of each word in the sentence patterns is calculated and stored in the form of a matrix.

In [46]:
# create training and output arrays
training = []
output_empty = [0] * len(labels)
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lem.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        if word in word_patterns:
            bag.append(1)
        else:
            bag.append(0)
            
    # copy output list
    output = list(output_empty)
    output[labels.index(document[1])] = 1
    training.append([bag, output])


## Preprocessing input and output data 

The training data is split into training input and training output. Also, the data is converted to an array format so that it 
can be inputted into the network.

In [47]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


C:\Users\aashi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


## Build Model

The next step is to build the actual model that will be trained with the preprocessed data. There are various factors to take into consideration when building the model, including the architecture and the parameter values. Ultimately, the model was continuously tested and modified until it registered a perfect accuracy. 

In [48]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

### Compile and Train Model

In [49]:
optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='acc')
model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1, callbacks=[checkpoint])

Epoch 1/500
140/140 [==============================] - 0s 2ms/sample - loss: 3.1217 - acc: 0.0357
Epoch 2/500
140/140 [==============================] - 0s 249us/sample - loss: 3.0664 - acc: 0.0643
Epoch 3/500
140/140 [==============================] - 0s 235us/sample - loss: 2.9963 - acc: 0.1071
Epoch 4/500
140/140 [==============================] - 0s 249us/sample - loss: 2.9599 - acc: 0.1286
Epoch 5/500
140/140 [==============================] - 0s 249us/sample - loss: 2.8833 - acc: 0.1500
Epoch 6/500
140/140 [==============================] - 0s 242us/sample - loss: 2.7961 - acc: 0.2000
Epoch 7/500
140/140 [==============================] - 0s 235us/sample - loss: 2.6801 - acc: 0.2000
Epoch 8/500
140/140 [==============================] - 0s 249us/sample - loss: 2.5900 - acc: 0.2786
Epoch 9/500
140/140 [==============================] - 0s 242us/sample - loss: 2.4805 - acc: 0.3143
Epoch 10/500
140/140 [==============================] - 0s 249us/sample - loss: 2.4206 - acc: 0.2714
E

140/140 [==============================] - 0s 221us/sample - loss: 0.1063 - acc: 0.9643
Epoch 83/500
140/140 [==============================] - 0s 221us/sample - loss: 0.1657 - acc: 0.9643
Epoch 84/500
140/140 [==============================] - 0s 214us/sample - loss: 0.0948 - acc: 0.9786
Epoch 85/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0976 - acc: 0.9857
Epoch 86/500
140/140 [==============================] - 0s 207us/sample - loss: 0.1381 - acc: 0.9714
Epoch 87/500
140/140 [==============================] - 0s 214us/sample - loss: 0.0749 - acc: 0.9929
Epoch 88/500
140/140 [==============================] - 0s 228us/sample - loss: 0.1025 - acc: 0.9786
Epoch 89/500
140/140 [==============================] - 0s 228us/sample - loss: 0.1035 - acc: 0.9714
Epoch 90/500
140/140 [==============================] - 0s 228us/sample - loss: 0.1262 - acc: 0.9643
Epoch 91/500
140/140 [==============================] - 0s 221us/sample - loss: 0.1269 - acc: 0.9714
Epo

140/140 [==============================] - 0s 235us/sample - loss: 0.1023 - acc: 0.9714
Epoch 162/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0453 - acc: 0.9929
Epoch 163/500
140/140 [==============================] - 0s 235us/sample - loss: 0.0621 - acc: 0.9929
Epoch 164/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0287 - acc: 0.9929
Epoch 165/500
140/140 [==============================] - 0s 214us/sample - loss: 0.0491 - acc: 0.9929
Epoch 166/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0362 - acc: 0.9929
Epoch 167/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0539 - acc: 0.9929
Epoch 168/500
140/140 [==============================] - 0s 235us/sample - loss: 0.0441 - acc: 0.9857
Epoch 169/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0410 - acc: 0.9857
Epoch 170/500
140/140 [==============================] - 0s 264us/sample - loss: 0.0384 - acc: 0

140/140 [==============================] - 0s 228us/sample - loss: 0.0266 - acc: 0.9929
Epoch 242/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0568 - acc: 0.9786
Epoch 243/500
140/140 [==============================] - 0s 242us/sample - loss: 0.0548 - acc: 0.9857
Epoch 244/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0429 - acc: 0.9929
Epoch 245/500
140/140 [==============================] - 0s 235us/sample - loss: 0.0195 - acc: 1.0000
Epoch 246/500
140/140 [==============================] - 0s 235us/sample - loss: 0.0346 - acc: 0.9929
Epoch 247/500
140/140 [==============================] - 0s 242us/sample - loss: 0.0491 - acc: 0.9786
Epoch 248/500
140/140 [==============================] - 0s 249us/sample - loss: 0.0344 - acc: 0.9929
Epoch 249/500
140/140 [==============================] - 0s 242us/sample - loss: 0.0393 - acc: 0.9929
Epoch 250/500
140/140 [==============================] - 0s 242us/sample - loss: 0.0380 - acc: 0

140/140 [==============================] - 0s 221us/sample - loss: 0.0337 - acc: 0.9929
Epoch 322/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0120 - acc: 1.0000
Epoch 323/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0442 - acc: 0.9857
Epoch 324/500
140/140 [==============================] - ETA: 0s - loss: 7.4881e-04 - acc: 1.000 - 0s 228us/sample - loss: 0.0645 - acc: 0.9786
Epoch 325/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0474 - acc: 0.9714
Epoch 326/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0281 - acc: 0.9857
Epoch 327/500
140/140 [==============================] - 0s 214us/sample - loss: 0.0964 - acc: 0.9714
Epoch 328/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0288 - acc: 0.9929
Epoch 329/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0510 - acc: 0.9857
Epoch 330/500
140/140 [==============================]

140/140 [==============================] - 0s 214us/sample - loss: 0.0404 - acc: 0.9929
Epoch 402/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0165 - acc: 0.9929
Epoch 403/500
140/140 [==============================] - 0s 214us/sample - loss: 0.0287 - acc: 0.9929
Epoch 404/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0174 - acc: 1.0000
Epoch 405/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0026 - acc: 1.0000
Epoch 406/500
140/140 [==============================] - 0s 235us/sample - loss: 0.0128 - acc: 1.0000
Epoch 407/500
140/140 [==============================] - 0s 242us/sample - loss: 0.0170 - acc: 0.9929
Epoch 408/500
140/140 [==============================] - 0s 242us/sample - loss: 0.0220 - acc: 1.0000
Epoch 409/500
140/140 [==============================] - 0s 256us/sample - loss: 0.0127 - acc: 1.0000
Epoch 410/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0327 - acc: 0

140/140 [==============================] - 0s 221us/sample - loss: 0.0198 - acc: 1.0000
Epoch 482/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0586 - acc: 0.9786
Epoch 483/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0125 - acc: 0.9929
Epoch 484/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0280 - acc: 0.9857
Epoch 485/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0108 - acc: 0.9929
Epoch 486/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0124 - acc: 1.0000
Epoch 487/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0510 - acc: 0.9857
Epoch 488/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0126 - acc: 1.0000
Epoch 489/500
140/140 [==============================] - 0s 228us/sample - loss: 0.0105 - acc: 1.0000
Epoch 490/500
140/140 [==============================] - 0s 221us/sample - loss: 0.0086 - acc: 1

In [50]:
best_model = load_model('best_model.h5')
best_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               15616     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 22)                1430      
Total params: 25,302
Trainable params: 25,302
Non-trainable params: 0
_________________________________________________________________


### Preprocess User Query

Now that the data is preprocessed and the model is built, it is time to test the efficacy of the model with user queries. However, the model won't be able to interpret user queries in its raw form. The queries will have to undergo the same preprocessing procedure as the text data used to train the model. That is the purpose of the functions shown below.

In [51]:
# performs preprocessing on inputted text (lower casing and lemmatization)
def preprocess_input(text):
    text_token = nltk.word_tokenize(text)
    text_token = [lem.lemmatize(word.lower()) for word in text_token]
    return text_token

# coverts preprocessed text to a matrix form 
def bow(text_token):
    bag = []
    for word in words:
        if word in text_token:
            bag.append(1)
        else:
            bag.append(0)
    return bag

# inputs user query in matrix form and returns a predicted label 
def predict_label(bag):
    prediction = best_model.predict(np.array(bag).reshape((1, len(words))))
    max_value = np.max(prediction)
    if max_value > 0.7:
        predict_label = labels[np.argmax(prediction)]
    else:
        predict_label = 'NA'
    return predict_label



## Response Generation

Once the model predicts the user's intent, it has to generate the appropriate response. Each intent has two responses, with one in English and the other in Spanish. 

In [52]:
responses = { 
        'cancel' : {
            'en':"Ok, it's cancelled. Once you're ready, please type *`daily`* to fill out your follow-up reports.",

            'es': "Ok, has cancelado tu seguimiento diario. Por favor escribe *`daily`* para empezar cuando desees."
        },
        'chicken' : {
            'en': "Here comes Arthur!\n\n*[Launch Rubber Chicken]({{{url}}})* 🐔\n\n_Pro tip: you can change the last part of Arthur's URL in your browser in order to create a unique room.",

            'es': "¡Aquí viene Arthur!\n\n[Abrir Rubber Chicken]({{{url}}}) 🐔\n\n_Pro tip: puedes cambiar la última parte de la URL de Arthur en tu navegador y así tendrás una sala única._"

        },
        'daily' : {
            'en':  "No response",
            'es':  "No hay respuesta"
        },
        'dailystatus' : {
            'en': "No response",
            'es': "No hay respuesta"
        },
        'dashboard' : {
            'en': "Ok, please *[follow this link]({{{url}}})* to access the web dashboard.",

            'es': "Ok, *[abre este enlace]({{{url}}})* para abrir la web y dashboard."

        },
        'datetime' : {
            'en':  "The time information for your user is:",
            'es': "La información horaria de tu usuario es:"

        },
        'disable' : {
            'en': "Ok, it's done. You have disconnected and disabled all private messaging between you and me.\nYou can still use the [web application]({{{url}}}).\n\nIf you want to talk to me again, just type *`enable`*",

            'es': "Listo, está hecho. Has desconectado y deshabilitado todos los mensajes privados entre tu y yo.\nPor cierto, puedes seguir utilizando mi [aplicación web]({{{url}}}).\n\nSi deseas volver a hablar conmigo simplemente escribe *`enable`*."

        },
        'enable' : {
            'en': "It seems that you had enabled me previously, I'm ready to help you.\nPlease type *`help`* to know more about how I can help you.",
                    # 8. "Hey! Welcome back, I'm ready to help you.\nYou can *[login into the web application]({{{url}}})* to check your settings."
            'es': "Parece que ya me habías habilitado previamente, estoy listo para ayudarte.\nPor favor, escribe *`help`* para saber más sobre cómo puedo ayudarte."
                    # 8. "¡Hola! Bienvenido de nuevo, estoy listo para ayudarte.\nPor favor inicia sesión en mi *[aplicación web]({{{url}}})* para confirmar su configuración."
        },
        'extras' : {
            'en': "Hey, your wish is my command, and these are the extra commands",

            'es': "Hola, puedes usar estos comandos extras para interactuar conmigo:"

        },
        'feedback' : {
            'en': "Hey *[user]*, it'd be great to get some feedback from you. If it's something important, I recommend you to visit our support center.",

            'es': "Hola *[user]*!, Sería grandióso recibir tu feedback. Si es algo importante, Te recomiendo visitar nuestro centro de soporte."

        },
        'hello' : {
            'en': "Hey [user]! Have a great day and remember I'm so happy about being your friend.\nType *`help`* to learn more about me.",

            'es': "¡Hola [user]! Ten un excelente día y recuerda que estoy muy feliz de ser tu amigo.\nEscribe *`help`* para aprender más sobre mí."

        },
        'help' : {
            'en': "Hey, your wish is my command, and these are my available commands:",

            'es': "Hola, puedes usar estos comandos para interactuar conmigo:"
        },
        'kudos' : {
            'en': "kudos response: ",
            'es':"kudos response: "
        },
        'news' : {
            'en': "Take a look at *[my blog](https://www.dailybot.co/blog/)* to read about tips, key learnings and other content that our team is sharing.",

            'es': "Echa un vistazo a *[mi blog](https://www.dailybot.com/blog/)* (en inglés) para leer sobre trucos, aprendizajes y otro contenido que comparte nuestro equipo."

        },
        'report' : {
            'en':'No response',
            'es':"No hay respuesta"
        },
        'resetdaily' : {
            'en': "This command has been changed, please type *`help`* to see all current commands.",

            'es': "Este comando ha cambiado, por favor escribe *`help`* para ver todos los comandos actuales."

        },
        'settings' : {
            'en': "This is a quick overview of your settings. You can manage these settings on my *[Web Application]({{{url}}})*:",

            'es': "Este es un resúmen de tu configuración. Puedes gestionar la configuración en mi *[aplicación web]({{{url}}})*:"

        },
        'skipdaily' : {
            'en':"no response",
            'es':"No hay respuesta"
        },
        'snooze' : {
            'en': "Ok, I’ll remind you in *{{time}} minutes*",

            'es': "Ok, te recordaré en *{{time}} minutos*."

        },
        'textformat' : {
            'en':"Hi there! The following is a list of the different formats I support.\nYou can also see this *[support article](https://www.dailybot.com)* to know more about how to build responses with these formats.",

            
            'es': "Hola! La siguiente es una lista de los diferentes formatos que soporto.\nPuedes verlo aquí *[artículo de apoyo](https://www.dailybot.com)* para conocer más acerca de como construir respuestas con estos formatos."

        },
        'thanks' : {
            'en': ["Oh, don't you even mention it. I'm always happy to help.",
                                "It's always my pleasure.",
                                "You are so very welcome.",
                                "It's no trouble at all. 😊",
                                "Anytime! it's always my pleasure. 😊"],
            'es': ["¡Con muuucho gusto!",
                                 "Siempre es un placer.",
                                 "Estoy para ayudarte.",
                                 "A ti por tu tiempo. 😊",
                                 "¡Cuando me necesites! siempre es un placer. 😊"]
        },
        'timezone' : {
            'en': "The time information for the typed timezone is:",

            'es': "La información de la zona horaria introducida es:"

        },
        'NA': {
            'en': "Sorry for not understanding. Please type *`help`* for me to help you.",
            'es': "Lo siento por no entender. Por favor escribe *`help`* para ayudarte."
        }
        
        
    }

In [53]:
# inputs the user intent and returns the appropriate response
def generate_response(query_intent):

        return responses[query_intent]

## Saving unidentified Queries

One way that can enhance the chatbot is to save user queries that can not be identified by the model. Since the model has a 100% accuracy, the main reason for it to not correctly identify an intent is that it is not trained by a sentence pattern that the user inputted. If certain sentences and phrases are used repeatedly without a proper response, these phrases need to be identified and added to the text data that will be used to retrain the model. 

There are various ways to implement this feature. For the sake of entertaining this idea, we will aim to achieve this funciton in a simple manner. We an create an empty .txt file called "no_intent_queries" that will store user queries that are not identified.

In [54]:
# Create list of queries with no response
try:
    no_intent_queries = open("no_intent_queries.txt", "x")
except:
    pass


In [55]:
queries_list = ""

## Chatbot

Here, we test the chatbot with the chatbot and evaluate its responses to the user queries. 

First, we create a while loop that runs ceaselessly untill the user types "quit".

Next, the previous preprocessing functions are applied to the user query, which is then run through the model to predict
the user intent. Finally, based on the predicted intent, an appropriate response is generated. This response will match the 
language of the initial query (i.e. English queries return English responses). 


In [56]:
print('Insert command here (type "quit" to stop chatbot)\n')

while 1 >0:
    
    input_text = input()
    if input_text=='quit':
        break
    if len(input_text) < 3:
        while len(input_text) <3:
            input_text = input_text + " "
    prediction = predict_label(np.array(bow(preprocess_input(input_text))))
    response = generate_response(prediction)
    if prediction == 'NA':
            queries_list = queries_list + input_text+'\n'    
    

    if TextBlob(input_text).detect_language() == 'en':
        if prediction == 'thanks':
            print(random.choice(response['en']))
        else:
            print(response['en'])
    elif TextBlob(input_text).detect_language() == 'es' :
        if prediction == 'thanks':
            print(random.choice(response['es'])+'\n')
        else:
            print(response['es']+'\n')
    else:
        print("Sorry for not understanding. Please type *`help`* for me to help you.")
    print(' ')
        

Insert command here (type "quit" to stop chatbot)

hello
Hey [user]! Have a great day and remember I'm so happy about being your friend.
Type *`help`* to learn more about me.
 
hi there
Hey [user]! Have a great day and remember I'm so happy about being your friend.
Type *`help`* to learn more about me.
 
hola
¡Hola [user]! Ten un excelente día y recuerda que estoy muy feliz de ser tu amigo.
Escribe *`help`* para aprender más sobre mí.

 
cancelar
Ok, has cancelado tu seguimiento diario. Por favor escribe *`daily`* para empezar cuando desees.

 
tablero
Ok, *[abre este enlace]({{{url}}})* para abrir la web y dashboard.

 
feedback please
Hey *[user]*, it'd be great to get some feedback from you. If it's something important, I recommend you to visit our support center.
 
quit


## Add "NA" Queries to .txt File

Once the user ceases use of the chatbot, all the queries that were not identified will be stored in the "no_intent_queries.txt" file. In theory, an employee can check upon this file regularly to see if certain queries stand out and should be added to the model. 

In [ ]:
with open("no_intent_queries.txt", "w") as file:
            file.write(queries_list)
            file.close()